In [1]:
import xlrd
import pandas as pd
from os import listdir
from os import path

In [2]:
def file_dataframe(excel_sheet, data_location):
    column_time = data_location['columns'][0]
    column_east = column_time + 1
    column_south = column_time + 2
    column_west = column_time + 3
    column_north = column_time + 4
    
    start_r = data_location['rows'][0] + 3
    end_r = data_location['rows'][1]
    
    times = excel_sheet.col_values(column_time,start_r,end_r)
    times_strip = [x.strip() for x in times]
    east = excel_sheet.col_values(column_east,start_r,end_r)
    south = excel_sheet.col_values(column_south,start_r,end_r)
    west = excel_sheet.col_values(column_west,start_r,end_r)
    north = excel_sheet.col_values(column_north,start_r,end_r)

    columns = ['times', 'east', 'south', 'west', 'north']
    return(pd.DataFrame({'times':times_strip, 'east':east, 'south':south, 'west':west, 'north':north}, columns=columns))

In [3]:
# finding date
def find_date(excel_sheet):
    sheet_c = excel_sheet.ncols
    sheet_r = excel_sheet.nrows
    date = ''
    for col in range(sheet_c):
        for row in range(sheet_r):
            cell_value = excel_sheet.cell_value(rowx=row, colx=col)
            if "date" in str(cell_value).lower():
                date = cell_value
    return date

In [4]:
def data_location(excel_sheet):
    sheet_c = excel_sheet.ncols
    sheet_r = excel_sheet.nrows
    start_c = 0
    start_r = 0
    end_c = sheet_c - 2
    end_r = sheet_r - 2
    for col in range(sheet_c):
        for row in range(sheet_r):
            cell_value = excel_sheet.cell_value(rowx=row, colx=col)
            if "time" in str(cell_value).lower():
                start_c = col
                start_r = row
    return pd.DataFrame.from_records([('start', start_c, start_r), ('end', end_c, end_r)], columns=['value', 'columns', 'rows'])

In [5]:
def find_address(excel_sheet):
    return excel_sheet.cell_value(rowx=0, colx=0)

In [20]:
data_directory = '../data/raw/TURNING MOVEMENT COUNT/'
data_file_names = listdir(data_directory)

In [21]:
def extract_data_sheet(sheet, sheet_name, sheet_data_location, counter):
    sheet_df = file_dataframe(sheet, sheet_data_location)
    sheet_df['data_id'] = counter
    return(sheet_df)

In [22]:
def log_data_sheet(sheet, sheet_name, sheet_data_location, counter, file_name):
    global data_info
    address = find_address(sheet)
    date = find_date(sheet)
    
    column_time = sheet_data_location['columns'][0]
    row_time = sheet_data_location['rows'][0]
    
    row_street = row_time + 1
    column_east = column_time + 1
    column_south = column_time + 2
    column_west = column_time + 3
    column_north = column_time + 4
    
    east = sheet.cell_value(row_street, column_east)
    south = sheet.cell_value(row_street, column_south)
    west = sheet.cell_value(row_street, column_west)
    north = sheet.cell_value(row_street, column_north)
    
    record = pd.DataFrame([(counter, address, date, east, south, west, north, sheet_name, file_name)], 
                          columns=['id','address','date','east','south','west','north','data_type','filename'])
    data_info = data_info.append(record)

In [23]:
def extract_and_log_data_sheet(workbook, sheet_name, counter, file_name):
    sheet_index = sheet_names.index(sheet_name)
    sheet = workbook.sheet_by_index(sheet_index)
    sheet_data_location = data_location(sheet)
    
    data_sheet = extract_data_sheet(sheet, sheet_name, sheet_data_location, counter)
    log_data_sheet(sheet, sheet_name, sheet_data_location, counter, file_name)
    
    return(data_sheet)

In [43]:
all_data = pd.DataFrame()
data_info = pd.DataFrame(columns=['id','address','date','east','south','west','north','data_type', 'filename'])

i = 0

for file_name in data_file_names:   
    if file_name.endswith('.XLS'):
        file_path = path.join(data_directory, file_name)
        workbook = xlrd.open_workbook(file_path)
        sheet_names = [x.lower() for x in workbook.sheet_names()]

        if 'all motors' in sheet_names:
            i = i + 1
            motor = extract_and_log_data_sheet(workbook, 'all motors', i, file_name)
            all_data = all_data.append(motor)
            
        if 'all peds hr.' in sheet_names:
            i = i + 1
            pedestrian = extract_and_log_data_sheet(workbook, 'all peds hr.', i, file_name)
            all_data = all_data.append(pedestrian)
        
        if 'bicycles hr.' in sheet_names:
            i = i + 1
            pedestrian = extract_and_log_data_sheet(workbook, 'bicycles hr.', i, file_name)
            all_data = all_data.append(pedestrian)

all_data.reset_index(drop=True, inplace=True)    
data_info.reset_index(drop=True, inplace=True)

all_data = all_data.apply(pd.to_numeric, errors='ignore')
data_info = data_info.apply(pd.to_numeric, errors='ignore')

all_data.to_csv(path_or_buf=data_directory + 'all_data.csv', index=False)
data_info.to_csv(path_or_buf=data_directory + 'data_info.csv', index=False)

In [11]:
data_info

,id,address,date,east,south,west,north,data_type,filename
0,1.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,all motors,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."
1,2.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,all peds hr.,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."
2,3.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,bicycles hr.,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."


In [12]:
all_data

,times,east,south,west,north,data_id
0,07:00-08:00,326.0,308.0,,670.0,1
1,08:00-09:00,331.0,344.0,,757.0,1
2,09:00-10:00,342.0,342.0,,752.0,1
3,10:00-11:00,350.0,279.0,,534.0,1
4,11:00-12:00,372.0,342.0,,545.0,1
5,12:00-13:00,390.0,348.0,,509.0,1
6,13:00-14:00,362.0,360.0,,627.0,1
7,14:00-15:00,449.0,392.0,,631.0,1
8,15:00-16:00,577.0,423.0,,580.0,1
9,16:00-17:00,562.0,398.0,,549.0,1


In [44]:
data_info.filename.nunique()

144

In [45]:
all_joined = pd.merge(left=all_data,right=data_info, left_on='data_id', right_on='id')

In [46]:
all_joined.groupby(['data_type']).sum()

,east_x,south_x,west_x,north_x,data_id,id
data_type,,,,,,
all motors,257657.0,265758.0,295610.0,312225.0,12174719,12174719.0
all peds hr.,59103.0,62762.0,57662.0,65817.0,300644,300644.0
bicycles hr.,3491.0,3541.0,4781.0,4651.0,303217,303217.0


In [47]:
data_info.head()

,id,address,date,east,south,west,north,data_type,filename
0,1.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,all motors,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."
1,2.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,all peds hr.,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."
2,3.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date -4.17.14,Dudley St.,Hampden St,Dunmore St.,Dudley St,bicycles hr.,"6813_207_DUDLEY-ST,-HAMPDEN-ST_NA_NA_ROXBURY_1..."
3,4.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date - 4.9.14,No. Beacon St.,Cambridge St.,Brighton Ave.,Cambridge St.,all motors,"6817_186_BRIGHTON-AVE,-CAMBRIDGE-ST,-NORTH-BEA..."
4,5.0,"Leo Sullivan 14 Dorr St. Roxbury, MA 02119 3500",Date - 4.9.14,No. Beacon St.,Cambridge St.,Brighton Ave.,Cambridge St.,all peds hr.,"6817_186_BRIGHTON-AVE,-CAMBRIDGE-ST,-NORTH-BEA..."
